### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [1]:
from unityagents import UnityEnvironment
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from ddpg_agent_p2 import Agent

In [2]:
env = UnityEnvironment(file_name='E:/code/deep-reinforcement-learning/p2_continuous-control/Reacher_Windows_x86_64/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# env_info = env.reset(train_mode=True)[brain_name]

# num_agents = len(env_info.agents)
# action_size = brain.vector_action_space_size
# state_size = env_info.vector_observations.shape[1]

In [5]:
def ddpg(n_episodes=1000, max_t=300, print_every=100):
    env_info = env.reset(train_mode=True)[brain_name]
    num_agents = len(env_info.agents)
    a_size = brain.vector_action_space_size
    s_size = env_info.vector_observations.shape[1]
    
    agent = Agent(state_size=s_size, action_size=a_size, random_seed=2)
    
    scores = []
    scores_deque = deque(maxlen = print_every)
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        state = states[0]
        agent.reset()
        score = 0
        while True:
            action = agent.act(state)
            env_info = env.step(action)[brain_name]
            next_states = env_info.vector_observations
            next_state = next_states[0]
            rewards = env_info.rewards
            reward = rewards[0]
            dones = env_info.local_done
            done = dones[0]
            score += reward
            agent.step(state, action, reward, next_state, done)
            state = next_state
            if np.any(dones):
                break
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tScore: {:.2f}'.format(i_episode, score), end="")
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque) >= 30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
        
    return scores

In [8]:
scores = ddpg(n_episodes=1000)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

UnityEnvironmentException: No Unity environment is loaded.

In [7]:
env.close()